In [1]:
import pandas as pd
# file name 합치기
from glob import glob
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from sklearn.model_selection import train_test_split
# 시각화
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from skimage import io, feature
from skimage.transform import resize
import cv2
from PIL import ImageChops

import seaborn as sns

# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

In [8]:
df = pd.read_csv("./데이터파일/노인의+선호여가+활동_20240904145858.csv")

In [9]:
df

,구분별(1),구분별(2),2013,2013.1,2013.2,2013.3,2013.4,2013.5,2013.6,2013.7,...,2018.9,2019,2019.1,2019.2,2019.3,2019.4,2019.5,2019.6,2019.7,2019.8
0,구분별(1),구분별(2),운동/건강 프로그램,노래/오락 프로그램,새로운 지식교육/교양 프로그램,수입과 연결된 직업관련 프로그램,여행 관광 등산 낚시 프로그램,사회봉사활동,사교 프로그램,전통문화 프로그램,...,없음,운동/건강 프로그램,노래/오락 프로그램,새로운 지식교육/교양 프로그램,수입과 연결된 직업관련 프로그램,여행 관광 등산 낚시 프로그램,사회봉사활동,사교 프로그램,전통문화 프로그램,없음
1,서울시,소계,52.8,8.9,7.5,6.1,4.2,3.6,1.5,1.5,...,11.9,54.6,9.6,7.7,3.9,8.5,3.2,2.3,1.3,8.9
2,성별,남자,52.6,5.9,9.7,7.2,4.6,3.5,1.1,1.7,...,10.0,57.1,6.9,9.1,4.4,9.0,2.8,1.9,0.7,8.0
3,성별,여자,53.1,11.5,5.6,5.1,3.9,3.7,1.8,1.3,...,13.5,52.6,11.7,6.6,3.5,8.0,3.6,2.7,1.7,9.5
4,연령별,만65세 이상,52.8,8.9,7.5,6.1,4.2,3.6,1.5,1.5,...,11.9,54.6,9.6,7.7,3.9,8.5,3.2,2.3,1.3,8.9
5,학력별,중졸 이하,53.0,10.6,8.4,6.4,3.9,3.4,1.1,1.3,...,14.3,53.4,10.8,6.2,4.0,8.0,3.1,2.2,1.1,11.1
6,학력별,고졸 이하,51.0,5.9,5.6,5.6,5.2,4.6,2.3,1.6,...,8.2,55.5,8.9,9.0,4.3,9.3,3.2,2.4,1.6,5.9
7,학력별,대졸 이하,59.2,4.8,5.4,4.2,3.0,2.0,1.7,2.8,...,10.4,57.0,5.6,12.3,2.2,8.1,4.3,2.6,0.9,7.1
8,학력별,대학원 이상,53.4,-,5.8,9.2,12.9,-,-,-,...,-,70.7,-,5.5,1.6,7.1,8.6,2.8,-,3.7
9,소득별,100만원 미만,36.8,15.7,8.2,9.2,6.0,3.9,2.8,2.6,...,17.9,58.2,12.5,2.7,4.2,4.5,2.1,0.8,1.0,13.9


In [3]:
df = df.T

In [4]:
df.columns = df.iloc[0]

In [7]:
df

구분별(1),구분별(1),서울시,성별,성별,연령별,학력별,학력별,학력별,학력별,소득별,...,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류
구분별(1),구분별(1),서울시,성별,성별,연령별,학력별,학력별,학력별,학력별,소득별,...,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류,지역소분류
구분별(2),구분별(2),소계,남자,여자,만65세 이상,중졸 이하,고졸 이하,대졸 이하,대학원 이상,100만원 미만,...,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구
2013,운동/건강 프로그램,52.8,52.6,53.1,52.8,53.0,51.0,59.2,53.4,36.8,...,53.1,51.9,65.3,54.8,41.2,68.1,44.0,62.6,61.6,47.2
2013.1,노래/오락 프로그램,8.9,5.9,11.5,8.9,10.6,5.9,4.8,-,15.7,...,15.9,3.4,10.0,10.1,7.5,3.6,16.5,4.6,5.8,6.6
2013.2,새로운 지식교육/교양 프로그램,7.5,9.7,5.6,7.5,8.4,5.6,5.4,5.8,8.2,...,1.5,22.7,4.8,5.5,9.8,4.1,12.9,6.9,8.8,28.6
2013.3,수입과 연결된 직업관련 프로그램,6.1,7.2,5.1,6.1,6.4,5.6,4.2,9.2,9.2,...,2.1,13.3,1.1,0.7,5.8,4.2,7.1,6.7,9.6,6.5
2013.4,여행 관광 등산 낚시 프로그램,4.2,4.6,3.9,4.2,3.9,5.2,3.0,12.9,6.0,...,5.3,2.3,6.4,4.8,4.0,5.9,2.2,3.6,2.9,3.9
2013.5,사회봉사활동,3.6,3.5,3.7,3.6,3.4,4.6,2.0,-,3.9,...,1.5,-,-,7.6,1.2,-,5.3,1.7,6.7,4.4
2013.6,사교 프로그램,1.5,1.1,1.8,1.5,1.1,2.3,1.7,-,2.8,...,0.4,-,1.0,1.8,9.5,-,2.2,1.1,-,1.2
2013.7,전통문화 프로그램,1.5,1.7,1.3,1.5,1.3,1.6,2.8,-,2.6,...,2.2,2.3,0.8,1.1,3.4,1.0,3.7,0.7,1.5,-


In [6]:
df.loc['구분별(1)']

구분별(1)
구분별(1)    구분별(1)
서울시          서울시
성별            성별
성별            성별
연령별          연령별
학력별          학력별
학력별          학력별
학력별          학력별
학력별          학력별
소득별          소득별
소득별          소득별
소득별          소득별
소득별          소득별
소득별          소득별
소득별          소득별
혼인상태별      혼인상태별
혼인상태별      혼인상태별
혼인상태별      혼인상태별
혼인상태별      혼인상태별
혼인상태별      혼인상태별
지역대분류      지역대분류
지역대분류      지역대분류
지역대분류      지역대분류
지역대분류      지역대분류
지역대분류      지역대분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
지역소분류      지역소분류
Name: 구분별(1), dtype: object